# 32-인스타그램 이미지 수집

## 모듈

In [1]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

import requests        # 이미지 개별 다운로드를 위해서 사용

# 파이썬에 내장되어 있는 패키지
import datetime as dt  # 생성할 폴더 이름에 날짜를 적용하기 위해 사용
import os              # 폴더를 생성하기 위해 사용
import zipfile         # 파일 압축을 위해 사용
import shutil          # 하위 파일이 존재하는 폴더를 삭제하기 위해 사용
import time            # 딜레이 적용을 위해 사용

## 크롬 브라우저 열기

In [2]:
# 크롬드라이버 자동 설치
chromedriver_autoinstaller.install()

'c:\\users\\junhong\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\chromedriver_autoinstaller\\91\\chromedriver.exe'

In [3]:
options = webdriver.ChromeOptions()
mobile_emulation = {"deviceName": "Nexus 5"}
options.add_experimental_option("mobileEmulation", mobile_emulation)

In [4]:
# 맥이나 리눅스의 경우 파일 확장자가 없다. 윈도우의 경우 exe 확장자까지 명시해야 한다.
driver = webdriver.Chrome(chrome_options=options)
# 모든 동작마다 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

<ipython-input-4-188c0386a583>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


## 이미지 수집

In [5]:
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)

In [38]:
# "#user_id"를 찾아서 id_input에 리턴할 때 까지 최대 3초간 무한반복
id_input = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("input[name='username']"))

# 찾은 요소에게 키보드 입력을 전달
id_input.send_keys("01084323575")

In [39]:
# "#user_id"를 찾아서 id_input에 리턴할 때 까지 최대 3초간 무한반복
id_input = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("input[name='password']"))

# 찾은 요소에게 키보드 입력을 전달
id_input.send_keys("sl237&46")

In [40]:
login_button = WebDriverWait(driver, 3).until(
                lambda x: x.find_element_by_css_selector("button[type='submit']"))
login_button.click()

In [41]:
time.sleep(5)

## 수집을 원하는 페이지로 이동하기
인스타그램의 페이지 URL은 아래와 같은 형식을 띄고 있다. 특정 사용자의 feed인지, 특정 해시태그에 대한 피드인지를 구분하여 URL을 지정할 수 있다.

- https://www.instagram.com/아이디/feed
- https://www.instagram.com/explore/tags/태그

In [42]:
driver.get("https://www.instagram.com/rachel_mypark/feed")
time.sleep(5)

In [29]:
# 이미지 목록을 저장할 빈 리스트
img_list = []
prev_list = None

page = 0

while True:
    
    # 현재 브라우저에 표시되고 있는 소스코드 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # srcset이라는 속성을 포함하는 모든 이미지 태그 가져오기 --> 리스트형으로 반환됨
    img = soup.select("img[srcset]")
    
    if prev_list == img:
        break
        
    
    # 미리 준비한 리스트에 결합시킴 ==> 중복되는 데이터가 발생할 수 있다.
    img_list += img
    
    # 스크롤을 맨 아래로 이동
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
        
    prev_list = img
    page += 1
    
    # 수집 과정을 출력한다.
    print("%04d번째 페이지에서 %02d건 수집함 >> 누적 데이터수: %05d" % (page, len(img), len(img_list)))

driver.quit()
img_list

0001번째 페이지에서 74건 수집함 >> 누적 데이터수: 00074
0002번째 페이지에서 59건 수집함 >> 누적 데이터수: 00133
0003번째 페이지에서 53건 수집함 >> 누적 데이터수: 00186
0004번째 페이지에서 47건 수집함 >> 누적 데이터수: 00233


[<img alt="" class="_7A2D8" src="https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/193949039_867659723818819_2059136287841364861_n.jpg?tp=1&amp;_nc_ht=scontent-gmp1-1.cdninstagram.com&amp;_nc_cat=107&amp;_nc_ohc=CMwXvdkHhUMAX-iekRX&amp;tn=2zck3XjWl2x2xrKe&amp;edm=ABZsPhsBAAAA&amp;ccb=7-4&amp;oh=d0ebc150d067cb9d8d5dc854c8c9eb68&amp;oe=60BF950F&amp;_nc_sid=4efc9f&amp;ig_cache_key=MjU4NzUxMjg4Nzg4NjE3MTU4MQ%3D%3D.2-ccb7-4"/>,
 <img alt="Photo by Colorful Travel Story 📸🇰🇷 in Hallstatt, Austria with @visitaustria_kr. May be an image of lake, nature and text that says 'Hallstatt Colorfu Travel Story'." class="FFVAD" crossorigin="anonymous" src="https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/193949039_867659723818819_2059136287841364861_n.jpg?tp=1&amp;_nc_ht=scontent-gmp1-1.cdninstagram.com&amp;_nc_cat=107&amp;_nc_ohc=CMwXvdkHhUMAX-iekRX&amp;tn=2zck3XjWl2x2xrKe&amp;edm=ABZsPhsBAAAA&amp;ccb=7-4&amp;oh=d0ebc150d067cb9d8d5dc854c8c9eb68&amp;oe=60BF950F&amp;_nc_sid=4efc9f&amp;

In [31]:
# 이미지의 주소만을 담기 위한 빈 리스트 준비
src_list = []

for t in img_list:
    srcset = t.attrs['src']      # srcset 속성 가져오기
    #srcset_list = srcset.split(",") # 쉼표 단위로 잘라서 리스트로 변환
    #item = srcset_list[-1]          # 이미지 해상도가 가장 큰 마지막 원소를 선택
   # p = item.find(" ")              # 공백문자의 위치 얻기
    url = srcset #item[:p]                  # 첫 번째 글자부터 마지막 공백문자 전까지 잘라냄
    src_list.append(url)            # 준비한 리스트에 추출결과 넣기
    
# 중복제거를 위해 집합으로 변경 후 리스트로 다시 변환
src_list = list(set(src_list))
src_list

['https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/195991126_337354057912979_8304840228290184889_n.jpg?tp=1&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=Z_zC8vl6VmcAX-G_JCi&tn=2zck3XjWl2x2xrKe&edm=ABZsPhsBAAAA&ccb=7-4&oh=ad660a4eec91fac5e89ab72acc2c880d&oe=60BFCC6F&_nc_sid=4efc9f&ig_cache_key=MjU4ODIzODUwODI4OTY0MjUxNw%3D%3D.2-ccb7-4',
 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/c0.0.896.896a/195164615_381888899899870_9026253729462219960_n.jpg?tp=1&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=lQzry-p7kxUAX9Nu_Ct&edm=ABZsPhsBAAAA&ccb=7-4&oh=a819bf7768cc1a0234efd916debf814c&oe=60C0F8D1&_nc_sid=4efc9f&ig_cache_key=MjU4ODE3OTEwNjAzMDIyNTEyMQ%3D%3D.2-ccb7-4',
 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/196033026_162072085931517_7131887194775408118_n.jpg?tp=1&_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=HlHRtOOn7X0AX8OwZhh&edm=AMKDjl4BAAAA&ccb=7-4&oh=70cddbb50a96d8d0f39a75a8106823a1&oe=60C0FD58&_nc_sid=1

In [32]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
header_info = {'User-agent': user_agent, 'referer': None}
header_info

{'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
 'referer': None}

In [33]:
# 저장된 이미지 수를 카운트하기 위한 변수
count = 0

# 수집된 이미지가 저장될 폴더이름 구성
datetime = dt.datetime.now().strftime("%y%m%d_%H%M%S")
dirname = "인스타그램_이미지수집_%s" % datetime

# 이미지를 저장할 폴더 만들기
if not os.path.exists(dirname):
    os.mkdir(dirname)
    
# 폴더 이름으로 빈 압축파일 생성
zipfile_name = '%s.zip' % dirname
insta_zip = zipfile.ZipFile(zipfile_name, 'w')

# 이미지 URL수 만큼 반복
for image_url in src_list:
    
    if "http" not in image_url:
        continue
        
    # 카운트 증가
    count += 1

    # 파일이 저장될 경로 생성  ==> 인스타그램_이미지수집_20201114_1420/0001.jpg
    path = "%s/%04d.jpg" % (dirname, count)

    # 예외처리를 동반한 이미지 다운로드
    try:
        # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
        r = requests.get(image_url, headers=header_info, stream=True)

        # HTTP 상태코드가 성공을 의미하는 값이 아니라면?
        if r.status_code != 200:
            # 에러코드와 에러메시지를 문자열로 구성
            err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
            print(err_msg)
            continue
            
        # 추출한 데이터를 바이너리(이진값) 쓰기 모드로 저장 -> 'wb'
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print( "[Ok] %s(이)가 저장되었습니다." % path )
        
        # 압축파일에 다운로드 된 이미지 추가
        insta_zip.write(path)
        
    except Exception as e:
        # 이미지 다운로드 실패시 에러가 발생함을 알림
        print( "[Error] `%s`의 저장에 실패했습니다." % path )
        # 개발자가 원인을 파악할 수 있도록 상세 에러 원인 출력
        print(e)
        # 다음 이미지를 시도하기 위해 반복의 조건식으로 이동함
        continue
        
# 압축파일 닫기
insta_zip.close()
print( "%s 압축파일이 생성되었습니다." % zipfile_name )

# 이미지가 수집된 폴더 삭제
shutil.rmtree(dirname)
print( "수집 폴더가 삭제되었습니다." )

[Ok] 인스타그램_이미지수집_210604_111709/0001.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0002.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0003.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0004.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0005.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0006.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0007.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0008.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0009.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0010.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0011.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0012.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0013.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0014.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0015.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0016.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0017.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0018.jpg(이)가 저장되었습니다.
[Ok] 인스타그램_이미지수집_210604_111709/0019.jpg(이)가 저장